Figure(s) in the manuscript created by this notebook: Fig. 6C, 6D.

This notebook takes the manually annotated cell outlines from the IRE1 photoconversion experiment and plots some basic statistics from them.

In [ ]:
# User-defined parameters for analysis:

# Plotting and figure saving params
save_figs = False
plot_settings = '../src/plotting_settings.py'
save_dir = '../reports/figures/Fig6_IRE1_photoconversion'

# Parameters for loading data from images manually measured in ImageJ.
# Each csv file represents one cell and should contain 
# four measurements in the following order:
# 1) Cell outline at the start of the experiment,
# 2) Cell outline before clusters begin to dissolve,
# 3) Cell outline right after clusters dissolve,
# 4) A region of comparable area to the cell to be used for background correction.

# Path to csv data
data_dir = '../data/processed/Fig6_IRE1_photoconv_2019-04-10/ManualOverlays'

frame_time = 10 # Duration of each frame, in minutes. Used for time scaling later
stress_start_min = 150 # time, in minutes, of ER stressor addition by the first frame

In [ ]:
# load the built-in and custom modules

# uncomment for debugging
"""
%load_ext autoreload 
%autoreload 2
"""

import os, sys, inspect
import glob
import matplotlib
import matplotlib.pylab as plt
import numpy as np
from scipy import stats
import pandas as pd
import seaborn as sns

# Add source code directory (src) to path to enable module import
module_dir = '../src'
os.sys.path.insert(0, module_dir)


In [ ]:
# Set up figure save dirs and load plotting style
if save_figs:
    %matplotlib
    %run $plot_settings save
    
    # Make directory for saving figures
    save_dir_pdf = os.path.join(save_dir, 'pdf')
    if not os.path.exists(save_dir_pdf):
        os.makedirs(save_dir_pdf)
else:
    %matplotlib inline
    %run $plot_settings plot_only

In [ ]:
# Load data

result_files = sorted(glob.glob(os.path.join(data_dir,'*.csv')))

filenames_no_ext = []
data_by_file = []

# Go file by file and read data
for i, file in enumerate(result_files):
    
    # Read data from the provided source file
    data = pd.read_csv(file) #, index_col=0)
    data.rename(columns={ data.columns[0]: "Meas_Type" }, inplace=True)
    filename_no_ext = os.path.split(os.path.splitext(file)[0])[1]
    data['file_name'] = filename_no_ext
    data['Cell_ID'] = i
    data_by_file.append(data)
    filenames_no_ext.append(filename_no_ext)
    
raw_data = pd.concat(data_by_file)
raw_data.reset_index(inplace=True, drop=True)
raw_data['Meas_Type'].replace(to_replace=1, value ="Start", inplace=True)
raw_data['Meas_Type'].replace(to_replace=2, value ="Pre_dis", inplace=True)
raw_data['Meas_Type'].replace(to_replace=3, value ="Post_dis", inplace=True)
raw_data['Meas_Type'].replace(to_replace=4, value ="Bkgnd", inplace=True) 

In [ ]:
# Create new dataframe where relevant properties are grouped by cell

cells = raw_data.loc[raw_data['Meas_Type'] == 'Start', 
                     ['Cell_ID', 'file_name']].copy()
cells.reset_index(inplace=True, drop=True)

# Subtract background and perform basic measurements
for measurement in raw_data['Meas_Type'].unique():
    data = raw_data.loc[raw_data['Meas_Type'] == measurement].copy()
    bkgnd = raw_data.loc[raw_data['Meas_Type'] == 'Bkgnd'].copy()
    data.reset_index(inplace=True, drop=True)
    bkgnd.reset_index(inplace=True, drop=True)
    
    corrected_intensity = data['Mean'] - bkgnd['Mean']
    
    int_col = 'Mean_' + measurement + '_bc'
    area_col = 'Area_' + measurement
    sum_col = 'SumI_' + measurement
    time_col = 'Time_m_' + measurement
    
    cells[int_col] = corrected_intensity
    cells[area_col] = data['Area']
    cells[sum_col] = data['Area'] * corrected_intensity
    cells[time_col] = (data['Slice']-1) * frame_time + stress_start_min


In [ ]:
# Analyze and plot the data

# Create summary figure
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(3.4, 1.8))
fig.tight_layout(pad=2)


cells['NormI_Start'] = 1
cells['NormI_Pre_dis'] = cells['SumI_Pre_dis'] / cells['SumI_Start']
cells['NormI_Post_dis'] = cells['SumI_Post_dis'] / cells['SumI_Start']

cells['Frac_dissolved'] = cells['SumI_Post_dis'] / cells['SumI_Pre_dis']
cells['Dissol_duration'] = cells['Time_m_Post_dis'] -  cells['Time_m_Pre_dis']

times = pd.melt(cells, id_vars=['Cell_ID'], 
                value_vars=['Time_m_Start', 'Time_m_Pre_dis', 'Time_m_Post_dis'],
                var_name='Condition', value_name='Time_min')
intensities = pd.melt(cells, id_vars=['Cell_ID'], 
                value_vars=['NormI_Start', 'NormI_Pre_dis', 'NormI_Post_dis'],
                var_name = 'Int_condition', value_name='Total_Intensities')
timecourse = pd.concat([times, intensities], axis=1)

timecourse = timecourse.loc[:,~timecourse.columns.duplicated()]
timecourse['Cell_ID_str'] = 'Cell ID ' + timecourse['Cell_ID'].astype(str)

sns.lineplot(x="Time_min", y="Total_Intensities", hue="Cell_ID_str", data=timecourse, ax=axes[0],
            legend=False)
sns.scatterplot(x="Time_min", y="Total_Intensities", hue="Cell_ID_str", data=timecourse, ax=axes[0],
               size=5, legend=False, style='Condition')
axes[0].set_xlabel('Stress duration (min)')
axes[0].set_ylabel('Photoconverted IRE1 fluorescence')
axes[0].set_ylim(0,)

sns.boxplot(x='Condition', y='Total_Intensities', data=timecourse, linewidth=0.5, showfliers = False, 
            boxprops={'facecolor':'None'}, ax=axes[1])
sns.swarmplot(x='Condition', y='Total_Intensities', data=timecourse, zorder=0.5, 
              ax=axes[1], size=3)
axes[1].set_ylim(0,)
axes[1].set_ylabel(None)

plt.show() 

if save_figs:
    fig_filename_pdf = os.path.join(save_dir_pdf, 'Photoconversion_summary.pdf')
    plt.savefig(fig_filename_pdf)

plt.show()

In [ ]:
# Calculate fraction of retained IRE1 intensity after dissolution

frac_left = []

for cell in timecourse['Cell_ID'].unique():
    row_pre_dis = (timecourse['Cell_ID'] == cell) & (timecourse['Int_condition'] == 'NormI_Pre_dis')
    pre_dis = float(timecourse.loc[row_pre_dis,'Total_Intensities'])
    row_post_dis = (timecourse['Cell_ID'] == cell) & (timecourse['Int_condition'] == 'NormI_Post_dis')
    post_dis = float(timecourse.loc[row_post_dis,'Total_Intensities'])
    frac_l = post_dis/pre_dis
    frac_left.append(frac_l)
    
print(np.mean(frac_left), stats.sem(frac_left))
    